In [ ]:
import numpy as np
import MDAnalysis as mda
import nglview as nv
from sklearn.decomposition import PCA
import requests
from Bio.PDB import *

### Overall settings

In [ ]:
movav_resis = 3 # number of residues used to calculate moving averages for CA positions (must be 3,5 or 7)
vector_scale_factor = 10 
vector_width = 1.0 
vector_colors = [ [0,0,1], [1,0,0], [0,1,0] ]

### G protein details

Give the residue numbers for Galpha H5.13 to H5.23 (as used in DOI [10.1073/pnas.1820944116](https://doi.org/10.1073/pnas.1820944116)) and the Galpha chain

In [ ]:
h5_inds = [341,351] # Gi residue numbers
#h5_inds = [337,347] # Gi (6OY9, 6OYA)
#h5_inds = [346,356] # Gq (+7DFL)
#h5_inds = [233,243] # Gq (6WHA)
#h5_inds = [381,391] # Gs
#h5_inds = [371,381] # Gs (7D3S, 6GDG)
#h5_inds = [2381,2391] # Gs (6E67)
#h5_inds = [367,377] # Gs (7JJO)
gpro_chain = "A"

### GPCR details

Then you can either read the GPCR details manually by running this...

In [ ]:
pdb_filename = "PDB_filename.pdb"
gpcr_chain = "R"
gpcr_name = "uniprotname_variant"

OR fetch GPCR structure from PDB:

In [ ]:
pdb = "7DFL"

# Download the PDB file
pdbl = PDBList()
pdb_filename = pdbl.retrieve_pdb_file( pdb, file_format = "pdb" )

# Get the protein name from GPCRdb
url = 'https://gpcrdb.org/services/structure/' + pdb + '/'
response = requests.get(url)
protein_data = response.json()
gpcr_chain = protein_data['preferred_chain']
gpcr_name = protein_data['protein']
print( "gpcr_chain:", gpcr_chain )
print( "gpcr_name:", gpcr_name )

Now fetch the GPCR details from GPCRdb by running the following:

In [ ]:
# For each helix, get the start and end residue numbers from GPCRdb
url = 'https://gpcrdb.org/services/residues/' + gpcr_name + '/'
response = requests.get(url)
protein_data = response.json()
tm_endpoints = np.zeros((8,2))
helix_no = 0
for i in protein_data:
    generic_no = i['display_generic_number']
    if generic_no == None:
        continue
    expected_prefix = str(helix_no) + "."
    next_prefix = str(helix_no + 1) + "."
    sequence_no = i['sequence_number']
    if generic_no[:2] == next_prefix:
        tm_endpoints[helix_no,0] = sequence_no
        helix_no += 1
    if generic_no[1] == ".":
        tm_endpoints[helix_no-1,1] = sequence_no

### Run the analysis

In [ ]:
u = mda.Universe( pdb_filename )

# Open NGLView instance
view1 = nv.show_mdanalysis(u)
view1.remove_cartoon()
view1.remove_ball_and_stick()
view1.add_cartoon('protein',color='#00BB00', opacity=0.3)

# Fit a vector to residues H5.13 to H5.23 and plot it
h5_selection = "(segid %s) and (resnum %d-%d) and (name CA)" % ( gpro_chain, h5_inds[0], h5_inds[1] )
h5_CAs = u.select_atoms( h5_selection )
h5_PCA = h5_CAs.principal_axes()
h5_cog = h5_CAs.center_of_geometry()
view1.shape.add_arrow( ( h5_cog + vector_scale_factor * h5_PCA[2] ).tolist(), ( h5_cog - vector_scale_factor * h5_PCA[2] ).tolist(), vector_colors[0], vector_width )

# For each TM, fit a vector to CA's #4-9 from the extracellular side
tm_vectors = np.zeros((7,3))
bundle_resnum = ""
for i in range(7):
    
    # Get the indices for extracellular residues no. 4-9 
    extracell_index = 0
    tm_startres = tm_endpoints[i,0] + 3
    if i%2 != 0:
        extracell_index = -1
        tm_startres = tm_endpoints[i,1] - 8
    print( "For TM%d, using residues %d-%d" % ( i+1, tm_startres, tm_startres + 5 ) )
    tm_resnum = "(resnum %d-%d)" % ( tm_startres, tm_startres + 5 )
    if i > 0:
        bundle_resnum += " or "
    bundle_resnum += tm_resnum
    
    # Use CA positions from PDB file
    tm_CAs = u.select_atoms( "segid %s and %s and (name CA)" % ( gpcr_chain, tm_resnum ) )
    if len( tm_CAs ) < 6:
        continue
    tm_PCA = tm_CAs.principal_axes()
    principal_idx = 2
    tm_cog = tm_CAs.center_of_geometry()
    
    # Overwrite: Use moving average of three CAs
    tm_CAs = u.select_atoms( "segid %s and (resnum %d-%d) and (name CA)" % ( gpcr_chain, tm_startres - ( movav_resis - 1 )/2, tm_startres + 5 + ( movav_resis - 1 )/2 ) )
    tm_pos = tm_CAs.positions
    tm_pos_movav = np.zeros( (6,3) )
    for k in range(6):
        tm_pos_movav[k,:] = np.average( tm_pos[ k:(k+movav_resis-1), : ], axis = 0 )
    pca = PCA()
    pca.fit( tm_pos_movav )
    tm_PCA = pca.components_
    principal_idx = 0
    
    # Make sure the vectors are pointing in the same direction
    dist_to_endpoint = np.zeros(2)
    for k in [0,1]:
        dist_to_endpoint[k] = np.sum( np.power( tm_cog + np.power( -1, k) * tm_PCA[ principal_idx ] - tm_pos[ extracell_index ], 2 ) )
    if dist_to_endpoint[1] > dist_to_endpoint[0]:
        tm_PCA[0] *= -1
    tm_vectors[i,:] = tm_PCA[ principal_idx ]
    # Plot the TM vector
    view1.shape.add_arrow( ( tm_cog - vector_scale_factor * tm_vectors[i,:] ).tolist(), ( tm_cog + vector_scale_factor * tm_vectors[i,:] ).tolist(), vector_colors[1], vector_width )

# Now calculate the GPCR axis by summing the TM vectors
bundle_vector = np.sum( tm_vectors, axis = 0 )
bundle_vector /= np.linalg.norm( bundle_vector )
bundle_CAs = u.select_atoms( "segid %s and (%s) and (name CA)" % ( gpcr_chain, bundle_resnum ) )
bundle_cog = bundle_CAs.center_of_geometry()
view1.shape.add_arrow( ( bundle_cog - vector_scale_factor * bundle_vector ).tolist(), ( bundle_cog + 4 * vector_scale_factor * bundle_vector ).tolist(), vector_colors[2], vector_width )

# Calculate the angle between the H5 vector and the GPCR axis
angle = np.round( np.rad2deg( np.arccos( np.dot( bundle_vector, h5_PCA[2] ) ) ) )
print( "Angle: %d degrees" % np.min( [ angle, 180-angle ] ) )

# Show the NGLView instance
view1